In [1]:
import os
import logging
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K

W0603 11:56:37.079900 139923816810304 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
%matplotlib inline


logger = logging.getLogger()
logger.setLevel(logging.INFO)


tf.enable_resource_variables()

PARAMETERS

The values used in the base model of transformer were; num_layers=6, d_model = 512, dff = 2048. See the paper for all the other versions of the transformer.


In [3]:
BERT_MODEL_URL = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

SEQ_LEN = 4
MAX_EXAMPLE_LEN = 512

BATCH_SIZE = 1
BUFFER_SIZE = 1

num_layers = 4
d_model = 128
dff = 512
num_heads = 8

# input_vocab_size = tokenizer_pt.vocab_size + 2
# target_vocab_size = tokenizer_en.vocab_size + 2

dropout_rate = 0.1

In [4]:
from data.load import load_cnn_dailymail
from ops.session import initialize_vars


train_dataset, val_dataset, test_dataset, metadata = load_cnn_dailymail()  

I0603 11:56:44.930810 139923816810304 resolver.py:79] Using /tmp/tfhub_modules to cache modules.
I0603 11:56:44.932458 139923816810304 resolver.py:398] Downloading TF-Hub Module 'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'.
I0603 11:57:01.042830 139923816810304 resolver.py:122] Downloading https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1: 201.62MB
I0603 11:57:16.401795 139923816810304 resolver.py:122] Downloading https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1: 421.62MB
I0603 11:57:16.410063 139923816810304 resolver.py:122] Downloaded https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1, Total size: 421.86MB
I0603 11:57:16.411129 139923816810304 resolver.py:413] Downloaded TF-Hub Module 'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'.


Instructions for updating:
Colocations handled automatically by placer.


W0603 11:57:17.430890 139923816810304 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0603 11:57:19.749305 139923816810304 saver.py:1489] Saver not created because there are no variables in the graph to restore
I0603 11:57:20.387201 139923816810304 dataset_builder.py:526] No config specified, defaulting to first: cnn_dailymail/plain_text
I0603 11:57:20.388325 139923816810304 dataset_builder.py:177] Load pre-computed datasetinfo (eg: splits) from bucket.
I0603 11:57:20.666056 139923816810304 dataset_info.py:377] Loading info from GCS for cnn_dailymail/plain_text/0.0.2
I0603 11:57:21.425375 139923816810304 dataset_builder.py:236] Generating dataset cnn_dailymail (/home/ec2-user/tensorflow_datasets/cnn_dailymail/plain_text/0.0.2)


...

W0603 12:02:16.116000 139923816810304 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_datasets/core/file_format_adapter.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


I0603 12:02:28.204631 139923816810304 dataset_builder.py:675] Generating split validation
I0603 12:02:28.207461 139923816810304 file_format_adapter.py:271] Writing TFRecords


I0603 12:02:37.348178 139923816810304 dataset_builder.py:675] Generating split test
I0603 12:02:37.350035 139923816810304 file_format_adapter.py:271] Writing TFRecords


I0603 12:02:45.303634 139923816810304 dataset_builder.py:264] Skipping computing stats for mode ComputeStatsMode.AUTO.


Dataset cnn_dailymail downloaded and prepared to /home/ec2-user/tensorflow_datasets/cnn_dailymail/plain_text/0.0.2. Subsequent calls will reuse this data.


In [5]:
def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [6]:
with open(b'/tmp/tfhub_modules/5a395eafef2a37bd9fc55d7f6ae676d2a134a838/assets/vocab.txt') as f:
    data = f.readlines()


In [7]:
UNK_ID = 100
CLS_ID = 101
SEP_ID = 102
MASK_ID = 103

In [8]:
def mask_timestamp(x, i, mask_with):
    """
    Masks each word in the summary draft one by one with the [MASK] token
    At t-th time step the t-th word of input summary is
    masked, and the decoder predicts the refined word given other
    words of the summary.
    
    x :: (N, T)
    returrn :: (N, T)
    """

    N, T = tf.shape(x)[0], tf.shape(x)[1]

    left = x[:, :i]
    right = x[:, i+1:]
    
    mask = tf.ones([N, 1], dtype=x.dtype) * mask_with
    
    masked = tf.concat([left, mask, right], axis=1)

    
    return masked


In [9]:
def loss_function(pred, real):
    """
    Since the target sequences are padded,
    it is important to apply a padding mask when calculating the loss.
    """
    logits_draft_summary, logits_refined_summary = pred
    
    target_ids_ = label_smoothing(tf.one_hot(real, depth=self.vocab_size))

    # use right shifted target
    loss_draft = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_draft_summary, labels=target_ids_[:, 1:])

    # use non-shifted target (we want to predict the masked word)
    loss_refined = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_refined_summary, labels=target_ids_[:, :-1])

    loss = loss_draft + loss_refined    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))    
    mask = tf.cast(mask, dtype=loss_.dtype)
    
    loss *= mask

    return tf.reduce_mean(loss)  


warmup_steps = 4000
initial_lr = 0.0003


def noam_scheme(init_lr, global_step, warmup_steps=4000.):
    '''Noam scheme learning rate decay
    init_lr: initial learning rate. scalar.
    global_step: scalar.
    warmup_steps: scalar. During warmup_steps, learning rate increases
        until it reaches init_lr.
    '''
    step = tf.cast(global_step + 1, dtype=tf.float32)
    return init_lr * warmup_steps ** 0.5 * tf.minimum(step * warmup_steps ** -1.5, step ** -0.5)


In [10]:
import os
import json
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import backend as K
from tensorflow.python.keras.initializers import Constant

from layers.transformer import Encoder
from layers.transformer import Decoder
from layers.bert import BertLayer, BERT_MODEL_URL
from layers.transformer import Decoder

from ops.masking import create_masks
from ops.regularization import label_smoothing


def _embedding_from_bert():
    """
    Extract the preratined word embeddings from a BERT model
    Returns a numpy matrix with the embeddings
    """
    logger.info("Extracting pretrained word embeddings weights from BERT")
    
    bert = hub.Module(BERT_MODEL_URL, trainable=False, name="embeddings_from_bert_module")    
    
    with tf.Session() as sess:
        initialize_vars(sess)
        embedding_matrix = sess.run(bert.variable_map['bert/embeddings/word_embeddings'])
                
    logger.info(f"Embedding matrix shape '{embedding_matrix.shape}'")
    return embedding_matrix


class AbstractiveSummarization(tf.keras.Model):
    """
    Pretraining-Based Natural Language Generation for Text Summarization 
    https://arxiv.org/pdf/1902.09243.pdf
    """
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, seq_len, rate=0.1):
        super(AbstractiveSummarization, self).__init__()
        
        self.seq_len = seq_len
        self.vocab_size = vocab_size
    
        self.bert = BertLayer(seq_len=seq_len, d_embedding=d_model, trainable=False)
        
        embedding_matrix = _embedding_from_bert()
        
        self.embedding = tf.keras.layers.Embedding(
            vocab_size, d_model, trainable=False,
            embeddings_initializer=Constant(embedding_matrix)
        )
        
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, vocab_size, rate)
                
        self.final_layer = tf.keras.layers.Dense(vocab_size)
    
    def encode(self, ids, mask, segment_ids):
        # (batch_size, seq_len, d_bert)
        return self.bert((ids, mask, segment_ids))    
    
    def draft_summary(self, target_ids, enc_output, training, look_ahead_mask, padding_mask):
        
        # (batch_size, seq_len, d_bert)
        embeddings = self.embedding(target_ids)    

        # (batch_size, seq_len, d_bert), (_)            
        dec_output, attention_dist = self.decoder(embeddings, enc_output, training, look_ahead_mask, padding_mask)
        
        # (batch_size, seq_len, vocab_len)
        logits = self.final_layer(dec_output)
        
        # (batch_size, seq_len)
        preds = tf.to_int32(tf.argmax(logits, axis=-1))
        
        return logits, preds, attention_dist      
    
    def refined_summary(self, target_ids, target_mask, target_segment_ids, enc_output, draft_summary, training, padding_mask):
        
        dec_outputs, attention_dists = [], []

        for i in range(0, self.seq_len):
            
            # (batch_size, seq_len)
            target_ids = mask_timestamp(target_ids, i, MASK_ID)
            
            # (batch_size, seq_len, d_bert)
            context_vectors = self.bert((target_ids, target_mask, target_segment_ids))
            
            # (batch_size, seq_len, d_bert), (_)
            dec_output, attention_dist = self.decoder(
                context_vectors,
                enc_output,
                training,
                look_ahead_mask=None,
                padding_mask=padding_mask
            )
            
            # (batch_size, 1, vocab_len)
            dec_outputs += [dec_output[:,i:i+1,:]]
            attention_dists += [{k: v[:, i:i+1 :] for k, v in attention_dist.items()}]

        # (batch_size, seq_len, d_bert)            
        dec_outputs = tf.concat(dec_outputs, axis=1)
                
        # (batch_size, seq_len, vocab_len)
        logits = self.final_layer(dec_outputs)    
        
        # (batch_size, seq_len)
        preds = tf.to_int32(tf.argmax(logits, axis=-1))
        
        return logits, preds, attention_dists
    
    def call(self, inp, tar, training=True):
        
        # (batch_size, seq_len) x3
        input_ids, input_mask, input_segment_ids = inp
        
        # (batch_size, seq_len + 1) x3
        target_ids, target_mask, target_segment_ids = tar   
                
        # (batch_size, 1, 1, seq_len), (_), (batch_size, 1, 1, seq_len)
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(input_ids, target_ids[:, :-1])

        # (batch_size, seq_len, d_bert)
        enc_output = self.encode(input_ids, input_mask, input_segment_ids)
        
        # (batch_size, seq_len , vocab_len), (batch_size, seq_len), (_)
        logits_draft_summary, preds_draft_summary, draft_attention_dist = self.draft_summary(
            target_ids[:, :-1],
            enc_output,
            training,
            combined_mask,
            dec_padding_mask
        )
        
        # (batch_size, seq_len, vocab_len), (batch_size, seq_len), (_)
        logits_refined_summary, preds_refined_summary, refined_attention_dist = self.refined_summary(
            target_ids[:, :-1],
            target_mask[:, :-1],
            target_segment_ids[:, :-1],
            enc_output,
            preds_draft_summary,
            training,
            dec_padding_mask
        )
        
        return logits_draft_summary, logits_refined_summary
    
    
    def train(self, xs, ys):
        # (batch_size, seq_len + 1) x3
        target_ids, target_mask, target_segment_ids = ys
        
        # (batch_size, seq_len, vocab_len), (batch_size, seq_len, vocab_len)
        logits_draft_summary, logits_refined_summary = self(xs, ys, True)
        
        target_ids_ = label_smoothing(tf.one_hot(target_ids, depth=self.vocab_size))

        # use right shifted target
        loss_draft = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_draft_summary, labels=target_ids_[:, 1:])
        
        # use non-shifted target (we want to predict the masked word)
        loss_refined = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_refined_summary, labels=target_ids_[:, :-1])

        loss = loss_draft + loss_refined    

        mask = tf.math.logical_not(tf.math.equal(target_ids[:, :-1], 0))    
        mask = tf.cast(mask, dtype=loss.dtype)

        loss *= mask
        loss = tf.reduce_mean(loss)        

        global_step = tf.train.get_or_create_global_step()
        learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)
        optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.9, beta2=0.98, epsilon=1e-9)
        train_op = optimizer.minimize(loss, global_step=global_step)

        tf.summary.scalar('lr', learning_rate)
        tf.summary.scalar("loss", loss)
        tf.summary.scalar("global_step", global_step)

        summaries = tf.summary.merge_all()

        return loss, train_op, global_step, summaries          

    
model = AbstractiveSummarization(
    num_layers=8,
    d_model=768,
    num_heads=8,
    dff=2048,
    vocab_size=30522,
    seq_len=SEQ_LEN,
    rate=0.1
)


I0603 12:02:45.634706 139923816810304 <ipython-input-10-faad59ca8c1b>:24] Extracting pretrained word embeddings weights from BERT
I0603 12:02:51.962241 139923816810304 <ipython-input-10-faad59ca8c1b>:32] Embedding matrix shape '(30522, 768)'


In [11]:
iterator = train_dataset.make_initializable_iterator()
stream = iterator.get_next()
xs, ys = stream[:3], stream[3:]

loss, train_op, global_step, train_summaries = model.train(xs, ys)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0603 12:02:55.931257 139923816810304 saver.py:1489] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0603 12:03:01.199451 139923816810304 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:143: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


...

I0603 12:03:08.753609 139923816810304 saver.py:1489] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0603 12:03:12.692908 139923816810304 saver.py:1489] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0603 12:03:16.194387 139923816810304 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [12]:
from config import config


with tf.Session() as sess:
    
    summary_writer = tf.summary.FileWriter(config.LOGDIR, sess.graph)
    
    sess.run(iterator.initializer)
    
    initialize_vars(sess)
    
    _gs = sess.run(global_step)
    
    for i in range(2):
        
        _, _gs, _summary = sess.run([train_op, global_step, train_summaries])
        
        summary_writer.add_summary(_summary, _gs)
        
    for var in tf.trainable_variables():
        print(var)
        

NameError: name 'vat' is not defined

In [14]:
_summary

b'\n\t\n\x02lr\x15\x87\x97q4\n\x0b\n\x04loss\x15\x94\x91\xa5A\n\x14\n\rglobal_step_1\x15\x00\x00\x00@'

In [13]:

_gs

global_step = tf.train.get_or_create_global_step()

learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)

optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.9, beta2=0.98, epsilon=1e-9)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# train_op = optimizer.minimize(loss, global_step=global_step)

model.compile(
    optimizer=optimizer,
    loss=loss_function,
    metrics=[tf.keras.metrics.categorical_accuracy]
)

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

In [42]:
isinstance(model, tf.keras.Model)

True

In [39]:
with tf.Session() as sess:
    print(tf.trainable_variables())


[<tf.Variable 'Adam/iterations:0' shape=() dtype=int64>, <tf.Variable 'Adam/lr:0' shape=() dtype=float32>, <tf.Variable 'Adam/beta_1:0' shape=() dtype=float32>, <tf.Variable 'Adam/beta_2:0' shape=() dtype=float32>, <tf.Variable 'Adam/decay:0' shape=() dtype=float32>, <tf.Variable 'Adam_1/iterations:0' shape=() dtype=int64>, <tf.Variable 'Adam_1/lr:0' shape=() dtype=float32>, <tf.Variable 'Adam_1/beta_1:0' shape=() dtype=float32>, <tf.Variable 'Adam_1/beta_2:0' shape=() dtype=float32>, <tf.Variable 'Adam_1/decay:0' shape=() dtype=float32>, <tf.Variable 'Adam_2/iterations:0' shape=() dtype=int64>, <tf.Variable 'Adam_2/lr:0' shape=() dtype=float32>, <tf.Variable 'Adam_2/beta_1:0' shape=() dtype=float32>, <tf.Variable 'Adam_2/beta_2:0' shape=() dtype=float32>, <tf.Variable 'Adam_2/decay:0' shape=() dtype=float32>]


In [30]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()
model.fit(el, epochs=10, steps_per_epoch=30)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 6 arrays: [<tf.Tensor 'IteratorGetNext_1:0' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_1:1' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_1:2' shape=(?, ?) dtype=int64>, <tf.Tensor 'Iterato...

In [14]:
with tf.Session() as sess:
    initialize_vars(sess)    
    model.fit(train_dataset, epochs=1, steps_per_epoch=1)

TypeError: not all arguments converted during string formatting

In [10]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
#     tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = el
    print(x0.shape, x1.shape, x2.shape)
    print(y0.shape, y1.shape, y2.shape)
    
    a = model((x0,x1,x2), (y0,y1,y2), True)
    
    initialize_vars(sess)
#     for i in m.trainable_weights: print(i)
#     raise ValueError
    
    a = sess.run([a])
    print(a)    



(?, ?) (?, ?) (?, ?)
(?, ?) (?, ?) (?, ?)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0602 02:27:36.021267 140483075295040 saver.py:1489] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0602 02:27:37.085570 140483075295040 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:143: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.cast instead.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0602 02:27:49.526523 140483075295040 saver.py:1489] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0602 02:27:53.526570 140483075295040 saver.py:1489] Saver not created because there are no variables in the graph to restore


[20.578041]


In [ ]:
# tf.summary.scalar('lr', lr)
# tf.summary.scalar("loss", loss)
# tf.summary.scalar("global_step", global_step)

# summaries = tf.summary.merge_all()

warmup_steps = 4000
initial_lr = 0.0003

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = model(inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)


def noam_scheme(init_lr, global_step, warmup_steps=4000.):
    '''Noam scheme learning rate decay
    init_lr: initial learning rate. scalar.
    global_step: scalar.
    warmup_steps: scalar. During warmup_steps, learning rate increases
        until it reaches init_lr.
    '''
    step = tf.cast(global_step + 1, dtype=tf.float32)
    return init_lr * warmup_steps ** 0.5 * tf.minimum(step * warmup_steps ** -1.5, step ** -0.5)


global_step = tf.train.get_or_create_global_step()

learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)

optimizer = tf.train.AdamOptimizer(learning_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)


iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
#     tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = el
    print(x0.shape, x1.shape, x2.shape)
    print(y0.shape, y1.shape, y2.shape)
    
    loss_value, grads = grad(model, (x0,x1,x2), (y0,y1,y2))
    
    print(loss_value)
    print(grads)

    initialize_vars(sess)



In [14]:
in_id = tf.keras.layers.Input(shape=(MAX_EXAMPLE_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(MAX_EXAMPLE_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(MAX_EXAMPLE_LEN,), name="segment_ids")

warmup_steps = 4000
initial_lr = 0.0003

def noam_scheme(init_lr, global_step, warmup_steps=4000.):
    '''Noam scheme learning rate decay
    init_lr: initial learning rate. scalar.
    global_step: scalar.
    warmup_steps: scalar. During warmup_steps, learning rate increases
        until it reaches init_lr.
    '''
    step = tf.cast(global_step + 1, dtype=tf.float32)
    return init_lr * warmup_steps ** 0.5 * tf.minimum(step * warmup_steps ** -1.5, step ** -0.5)


global_step = tf.train.get_or_create_global_step()

learning_rate = noam_scheme(initial_lr, global_step, warmup_steps)

optimizer = tf.train.AdamOptimizer(learning_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer)

In [4]:
tmp = tf.constant([[[1,2,3,4], [5,6,7,8]]])

def loss_function(pred, real):
    """
    Since the target sequences are padded,
    it is important to apply a padding mask when calculating the loss.
    """
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=real)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)    



with tf.Session() as sess:
    print(sess.run())


Instructions for updating:
Colocations handled automatically by placer.


W0601 17:55:21.698049 140022164780864 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/resource_variable_ops.py:435: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


FailedPreconditionError: Error while reading resource variable global_step from Container: localhost. This could mean that the variable was uninitialized. Not found: Container localhost does not exist. (Could not find resource: localhost/global_step)
	 [[node global_step/Read/ReadVariableOp (defined at <ipython-input-4-76a7a9e69621>:3) ]]

Caused by op 'global_step/Read/ReadVariableOp', defined at:
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ec2-

The negative log likelihood loss. It is useful to train a classification problem with `C` classes.

Computes sparse softmax cross entropy between `logits` and `labels`. Measures the probability error in discrete classification tasks in which the
  classes are mutually exclusive (each entry is in exactly one class).

In [29]:
session = tf.Session()
print(session.run(tf.global_variables_initializer()))
print(session.run(tf.report_uninitialized_variables()))


None
[]


In [28]:
vector = tf.constant([[1, 2, 3, 4], [1,2,3,4]])
print(vector.shape)

with tf.Session() as sess:
    vectors = [vector[:, i:i+1] for i in range(4)]
    print(sess.run(tf.concat(vectors, axis=1)))
        
#     print(sess.run())


(2, 4)
[[1 2 3 4]
 [1 2 3 4]]


In [ ]:
from layers.bert import BertLayer


class BertLayer(tf.keras.layers.Layer):
    """
    Custom Keras layer, integrating BERT from tf-hub
    """
    def __init__(self, url, seq_len=512, d_embedding=768, n_fine_tune_layers=None, **kwargs):
        self.url = url
        self.n_fine_tune_layers = n_fine_tune_layers
        self.seq_len = seq_len
        self.d_embedding = d_embedding
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.url,
            trainable=self.trainable,
            name="{}_bert_module".format(self.name)
        )

        trainable_vars = self.bert.variables
        
        if self.trainable:

            # Remove unused layers
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

            # Select how many layers to fine tune
            trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

            # Add to trainable weights
            for var in trainable_vars:
                self._trainable_weights.append(var)

            for var in self.bert.variables:
                if var not in self._trainable_weights:
                    self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        
        input_ids, input_mask, segment_ids = inputs
        
        bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)["sequence_output"]
        return result

    def compute_output_shape(self, input_shape):
        raise ValueError()
        return (input_shape[0], self.seq_len, self.d_embedding)

    
    
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)

In [14]:
atmp1 = np.zeros((30522))
atmp1[10] = 1

atmp2 = np.zeros((30522))
atmp2[20] = 1

In [49]:
atmp = np.array([atmp1, atmp2], dtype=np.float32)
atmp.dtype

dtype('float32')

In [50]:
from tensorflow.python.keras.initializers import Constant


with tf.Session() as sess:
    
    initialize_vars(sess)
    embedding_matrix = sess.run(bert.variable_map['bert/embeddings/word_embeddings'])

target_vocab_size = len(data)
d_model = 768

embedding = tf.keras.layers.Embedding(target_vocab_size, d_model, trainable=False, embeddings_initializer=Constant(embedding_matrix))



In [59]:
embedding.compute_output_shape((10, target_vocab_size))

TensorShape([Dimension(10), Dimension(30522), Dimension(768)])

In [52]:
with tf.Session() as sess:
    
    initialize_vars(sess) 
    atmp_ = tf.convert_to_tensor(atmp, np.float32)
    print(atmp_)
    print(sess.run(embedding(atmp_)).shape)
    raise ValueError
    print(sess.run(tf.matmul(atmp, bert.variable_map['bert/embeddings/word_embeddings'])))
    print(sess.run(bert.variable_map['bert/embeddings/word_embeddings'](np.array([0, 1]))))


    

Tensor("Const_20:0", shape=(2, 30522), dtype=float32)


NotFoundError: Resource localhost/embedding_5/embeddings/N10tensorflow3VarE does not exist.
	 [[node embedding_5/embedding_lookup (defined at <ipython-input-52-fac9b5b8d76d>:6) ]]

Caused by op 'embedding_5/embedding_lookup', defined at:
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/ec2-user/anaconda3

In [53]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)
    initialize_vars(sess)    
    tmp = sess.run(el)
    x0, x1, x2, y0, y1, y2 = tmp
    print(x0.shape, x1.shape, x2.shape)
#     print(x0, '\n', x1)
    tmp2 = model.predict([x0, x1, x2])
    print(tmp2)
#     print(sess.run(tmp2))  # (batch_size, input_seq_len, d_model)    


    

(1, 512) (1, 512) (1, 512)


NameError: name 'model' is not defined

In [9]:
iter = train_dataset.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:

#     sess.run(iter.initializer)
#     tmp = sess.run(el)
#     print(tmp)
    sample_transformer = Transformer(
        num_layers=2, d_model=512, num_heads=8, dff=2048, 
        input_vocab_size=8500, target_vocab_size=8000)

    temp_input = tf.random.uniform((64, 62))
    temp_target = tf.random.uniform((64, 26))

    fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                                   enc_padding_mask=None, 
                                   look_ahead_mask=None,
                                   dec_padding_mask=None)
    initialize_vars(sess)
    
    print(sess.run(fn_out))  # (batch_size, input_seq_len, d_model)    
#     x0, x1, x2, y0, y1, y2 = tmp
#     tmp2 = model.predict([x0, x1, x2])
    

[[[ 0.49737298  0.5550515  -0.13341121 ... -0.5284789  -0.61509997
   -0.04155324]
  [ 0.5157517   0.5635947  -0.09799525 ... -0.5260042  -0.62772596
   -0.03477238]
  [ 0.53440964  0.5700865  -0.05099516 ... -0.5282975  -0.63185966
   -0.02259888]
  ...
  [ 0.39933348  0.4695823  -0.19357261 ... -0.49450147 -0.70106435
   -0.01530197]
  [ 0.41170758  0.47419053 -0.20090303 ... -0.5092494  -0.7242769
    0.0037366 ]
  [ 0.42569858  0.49202585 -0.1921086  ... -0.52926743 -0.73894566
    0.00663666]]

 [[ 0.49737298  0.5550515  -0.13341121 ... -0.5284789  -0.61509997
   -0.04155324]
  [ 0.5157517   0.5635947  -0.09799525 ... -0.5260042  -0.62772596
   -0.03477238]
  [ 0.53440964  0.5700865  -0.05099516 ... -0.5282975  -0.63185966
   -0.02259888]
  ...
  [ 0.39933348  0.4695823  -0.19357261 ... -0.49450147 -0.70106435
   -0.01530197]
  [ 0.41170758  0.47419053 -0.20090303 ... -0.5092494  -0.7242769
    0.0037366 ]
  [ 0.42569858  0.49202585 -0.1921086  ... -0.52926743 -0.73894566
    0.00

In [30]:
sess = tf.InteractiveSession()


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [6]:
def print_out(q, k, v):
    temp_out, temp_attn = scaled_dot_product_attention(q, k, v, None)
    print ('Attention weights are:')
    print (temp_attn.eval())
    print ('Output is:')
    print (temp_out.eval())

    
temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# This query aligns equally with the first and second key, 
# so their values get averaged.
temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
print_out(temp_q, temp_k, temp_v)




Attention weights are:


ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [7]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
print(out.shape, attn.shape)

sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512)))

NameError: name 'MultiHeadAttention' is not defined

models

In [29]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048, 
    input_vocab_size=8500, target_vocab_size=8000)

temp_input = tf.random.uniform((64, 62))
temp_target = tf.random.uniform((64, 26))

fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)


TensorShape([Dimension(64), Dimension(26), Dimension(8000)])

### Optimizer

In [31]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    """
    Use the Adam optimizer with a custom learning rate scheduler according to the formula
    """
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


AttributeError: module 'tensorflow._api.v1.keras.optimizers' has no attribute 'schedules'

### Losses and Metrics

In [43]:
loss_object = tf.keras.losses.sparse_categorical_crossentropy(from_logits=True)


TypeError: sparse_categorical_crossentropy() missing 2 required positional arguments: 'y_true' and 'y_pred'

In [41]:
def loss_function(real, pred):
    """
    Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.
    """
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')


In [44]:
def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask


In [34]:
x0.shape

(1, 512)

In [33]:
x0[:, :-1].shape


(1, 511)

In [25]:
sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, target_vocab_size=8000)

output, attn = sample_decoder(tf.random.uniform((64, 26)), 
                              enc_output=sample_encoder_output, 
                              training=False, look_ahead_mask=None, 
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape, attn['decoder_layer2_block1'].shape


(TensorShape([Dimension(64), Dimension(26), Dimension(512)]),
 TensorShape([Dimension(64), Dimension(8), Dimension(26), Dimension(62)]),
 TensorShape([Dimension(64), Dimension(8), Dimension(26), Dimension(26)]))

In [21]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, input_vocab_size=8500)

sample_encoder_output = sample_encoder(tf.random.uniform((64, 62)), 
                                       training=False, mask=None)

print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)


(64, 62, 512)


In [18]:
sample_decoder_layer = DecoderLayer(512, 8, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output, 
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)


TensorShape([Dimension(64), Dimension(50), Dimension(512)])

In [14]:
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0523 12:59:35.651068 140526081922880 saver.py:1489] Saver not created because there are no variables in the graph to restore


In [19]:
import tensorflow as tf


def create_padding_mask(seq):
    """
    Mask all the pad tokens in the batch of sequence.
    It ensures that the model does not treat padding as the input.
    The mask indicates where pad value 0 is present:
    it outputs a 1 at those locations, and a 0 otherwise.    
    """
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions so that we can add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)


def create_look_ahead_mask(size):
    """
    The look-ahead mask is used to mask the future tokens in a sequence.
    In other words, the mask indicates which entries should not be used. 

    This means that to predict the third word, only the first and second word will be used.
    Similarly to predict the fourth word, only the first, second and the third word will be used and so on.
    """
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


def create_masks(inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, combined_mask, dec_padding_mask



In [47]:
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])

with tf.Session() as sess:
    print(sess.run(create_padding_mask(x)).shape)


(3, 1, 1, 5)


In [48]:
create_masks(x0, y0)

(<tf.Tensor 'strided_slice_9:0' shape=(1, 1, 1, 512) dtype=float32>,
 <tf.Tensor 'Maximum_1:0' shape=(1, 1, 512, 512) dtype=float32>,
 <tf.Tensor 'strided_slice_10:0' shape=(1, 1, 1, 512) dtype=float32>)

In [42]:
x0[:, :-1]

array([[  101,  2028,  1997,  3725,  1005,  1055,  2087,  6450,  2406,
         5014,  2038,  2908,  2006,  1996,  3006,  1011,  1998,  2009,
         2428,  2003,  4906,  2005,  1037,  3035,  1012,  1996,  3694,
         1045,  3205,  2026, 13629,  2380,  1010,  2275,  1999,  1037,
         1015,  1010,  8380,  1011,  7456,  3776,  1010,  2038,  1037,
         2157,  2548,  3976,  6415,  1011,  1037, 22771, 14534,  2629,
         2213,  1012,  2021,  1996,  3776,  6074,  8304,  1996,  5096,
         2360,  2045,  2003,  2053, 15843,  1997,  4022, 17394,  5782,
         2000,  3193,  1996,  3200,  1012,  2188,  4906,  2005, 16664,
         1024,  2026, 13629,  2380,  2003,  3694,  1045,  3205,  1998,
         2001,  3294, 10601,  2058,  1037,  2093,  2095,  2622,  2011,
         1996,  2783,  5608,  1012,  1996,  3035,  2218,  1037, 12695,
         2182,  1999,  3541, 21136,  7430,  2005,  1017,  1010,  2199,
         6368,  1999,  2494,  1012,  2009,  2003,  2028,  1997,  3725,
      

In [15]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True)

train_examples, val_examples = examples['train'], examples['validation']


I0528 14:56:33.385675 139957305521984 dataset_builder.py:177] Load pre-computed datasetinfo (eg: splits) from bucket.
I0528 14:56:33.650062 139957305521984 dataset_info.py:377] Loading info from GCS for ted_hrlr_translate/pt_to_en/0.0.1
I0528 14:56:34.125548 139957305521984 dataset_builder.py:236] Generating dataset ted_hrlr_translate (/home/ec2-user/tensorflow_datasets/ted_hrlr_translate/pt_to_en/0.0.1)


I0528 14:56:34.203127 139957305521984 download_manager.py:241] Downloading http://www.phontron.com/data/qi18naacl-dataset.tar.gz into /home/ec2-user/tensorflow_datasets/downloads/phontron.com_qi18naacl-datasetLVhDyvf9PQ-GjP4jim31ESZuvRjHI5wpNhR5SJvsNOo.tar.gz.tmp.85acbe6968a948c0ab60989f292e5947...
I0528 14:56:46.151685 139957305521984 dataset_builder.py:675] Generating split train
I0528 14:56:46.155997 139957305521984 file_format_adapter.py:271] Writing TFRecords


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


W0528 14:56:52.763825 139957305521984 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow_datasets/core/file_format_adapter.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


I0528 14:56:53.236819 139957305521984 dataset_builder.py:675] Generating split validation
I0528 14:56:53.238343 139957305521984 file_format_adapter.py:271] Writing TFRecords


I0528 14:56:53.504857 139957305521984 dataset_builder.py:675] Generating split test
I0528 14:56:53.506487 139957305521984 file_format_adapter.py:271] Writing TFRecords


I0528 14:56:53.876027 139957305521984 dataset_builder.py:264] Skipping computing stats for mode ComputeStatsMode.AUTO.


Dataset ted_hrlr_translate downloaded and prepared to /home/ec2-user/tensorflow_datasets/ted_hrlr_translate/pt_to_en/0.0.1. Subsequent calls will reuse this data.


In [22]:
iter = train_examples.make_initializable_iterator()
el = iter.get_next()

with tf.Session() as sess:
    sess.run(iter.initializer)  
    x, y = sess.run(el)
    print(x.decode('utf-8'))
    print()
    print(y.decode('utf-8'))

os astrónomos acreditam que cada estrela da galáxia tem um planeta , e especulam que até um quinto deles tem um planeta do tipo da terra que poderá ter vida , mas ainda não vimos nenhum deles .

astronomers now believe that every star in the galaxy has a planet , and they speculate that up to one fifth of them have an earth-like planet that might be able to harbor life , but we have n't seen any of them .
